In [1]:
import numpy as np

In [2]:
#########################################################################################
#  Generator  Hyper-parameters
######################################################################################
EMB_DIM = 200 # embedding dimension
HIDDEN_DIM = 200 # hidden state dimension of lstm cell
MAX_SEQ_LENGTH = 17  # max sequence length
BATCH_SIZE = 64

In [3]:
#########################################################################################
#  Basic Training Parameters
#########################################################################################
TOTAL_BATCH = 2000
dataset_path = "../data/movie/"
emb_dict_file = dataset_path + "imdb_word.vocab"

# imdb corpus, id is the one hot translation of txt sentences
imdb_file_txt = dataset_path + "imdb/imdb_sentences.txt"
imdb_file_id = dataset_path + "imdb/imdb_sentences.id"

eval_file = 'save/eval_file.txt'
eval_text_file = 'save/eval_text_file.txt'
negative_file = 'save/generator_sample.txt'
infer_file = 'save/infer/'

In [4]:
def load_emb_data(emb_dict_file):
    word_dict = {}
    word_list = []
    item = 0
    with open(emb_dict_file, 'r') as f:
        lines = f.readlines()
        for line in lines:
            # remove /n
            word = line.strip()
            word_dict[word] = item
            item += 1
            word_list.append(word)
    length = len(word_dict)
    print("Load embedding success! Num: %d" % length)
    return word_dict, length, word_list

In [5]:
class Gen_Data_loader():
    def __init__(self, batch_size, vocab_dict):
        self.batch_size = batch_size
        self.token_stream = []
        self.vocab_size = 0
        self.vocab_dict = vocab_dict

    def create_batches(self, data_file_list):
        """make self.token_stream into a integer stream."""
        self.token_stream = []
        print("load %s file data.." % ' '.join(data_file_list))
        for data_file in data_file_list:
            with open(data_file, 'r') as f:
                for line in f:
                    line = line.strip()
                    # sentence into separate words
                    line = line.split()
                    parse_line = [int(x) for x in line]
                    # making all the words in the dataset as a 1d stream
                    self.token_stream.append(parse_line)

        self.num_batch = int(len(self.token_stream) / self.batch_size)
        # cut the taken_stream's length exactly equal to num_batch * batch_size
        self.token_stream = self.token_stream[:self.num_batch * self.batch_size]
        self.sequence_batch = np.split(np.array(self.token_stream), self.num_batch, 0)
        self.pointer = 0
        print("      Load %d * %d batches" % (self.num_batch, self.batch_size))

    def next_batch(self):
        """take next batch by self.pointer"""
        ret = self.sequence_batch[self.pointer]
        self.pointer = (self.pointer + 1) % self.num_batch
        return ret

    def reset_pointer(self):
        self.pointer = 0

In [6]:
class Dis_Data_loader():
    def __init__(self, batch_size, vocab_dict, max_sequence_length):
        self.batch_size = batch_size
        self.sentences = np.array([])
        self.labels = np.array([])
        self.vocab_dict = vocab_dict
        self.max_sequence_length = max_sequence_length

    def load_train_data(self, positive_file_list, negative_file_list):
        # Load data
        positive_examples = []
        negative_examples = []
        for positive_file in positive_file_list:
            with open(positive_file)as fin:
                for line in fin:
                    line = line.strip()
                    line = line.split()
                    parse_line = [int(x) for x in line]
                    positive_examples.append(parse_line)
        for negative_file in negative_file_list:
            with open(negative_file)as fin:
                for line in fin:
                    line = line.strip()
                    line = line.split()
                    parse_line = [int(x) for x in line]
                    negative_examples.append(parse_line)

        self.sentences = np.array(positive_examples + negative_examples)
        self.sentences = self.padding(self.sentences, self.max_sequence_length)

        # Generate labels
        positive_labels = [[0, 1] for _ in positive_examples]
        negative_labels = [[1, 0] for _ in negative_examples]
        self.labels = np.concatenate([positive_labels, negative_labels], 0)

        # Shuffle the data
        shuffle_indices = np.random.permutation(np.arange(len(self.labels)))
        self.sentences = self.sentences[shuffle_indices]
        self.labels = self.labels[shuffle_indices]

        # Split batches
        self.num_batch = int(len(self.labels) / self.batch_size)
        self.sentences = self.sentences[:self.num_batch * self.batch_size]
        self.labels = self.labels[:self.num_batch * self.batch_size]
        self.sentences_batches = np.split(self.sentences, self.num_batch, 0)
        self.labels_batches = np.split(self.labels, self.num_batch, 0)
        self.pointer = 0

    def next_batch(self):
        """take next batch (sentence, label) by self.pointer"""
        ret = self.sentences_batches[self.pointer], self.labels_batches[self.pointer]
        self.pointer = (self.pointer + 1) % self.num_batch
        return ret

    def reset_pointer(self):
        self.pointer = 0

    def padding(self, inputs, max_sequence_length):
        batch_size = len(inputs)
        inputs_batch_major = np.zeros(shape=[batch_size, max_sequence_length], dtype=np.int32)  # == PAD
        for i, seq in enumerate(inputs):
            for j, element in enumerate(seq):
                inputs_batch_major[i, j] = element
        return inputs_batch_major

In [7]:
# load embedding info
vocab_dict, vocab_size, vocab_list = load_emb_data(emb_dict_file)

Load embedding success! Num: 26401


In [8]:
# prepare data
pre_train_data_loader = Gen_Data_loader(BATCH_SIZE, vocab_dict)
pre_train_data_loader.create_batches([imdb_file_id])

load ../data/movie/imdb/imdb_sentences.id file data..
      Load 10926 * 64 batches


In [9]:
gen_data_loader = Gen_Data_loader(BATCH_SIZE, vocab_dict)
gen_data_loader.create_batches([imdb_file_id])

dis_data_loader = Dis_Data_loader(BATCH_SIZE, vocab_dict, MAX_SEQ_LENGTH)

load ../data/movie/imdb/imdb_sentences.id file data..
      Load 10926 * 64 batches
